# Projekt - Dynamic Inverse Kinematics

## Opis zagadnienia

Celem projektu jest rozwiązanie problemu Inverse Kinematics w dynamicznie zmieniającym się środowisku. W klasycznej wersji problemu dane jest ramię złożone z pewnej liczby segmentów. Celem jest takie dobranie kątów między segmentami, aby koniec ostatniego segmentu znalazł się w zadanym punkcie. W moim projekcie rozważam wersję problemu, w której obecne są dodatkowo prostokątne przeszkody. Co więcej, zarówno położenie przeszkód jak i celu może zmieniać się wraz z czasem.

## Opis metody

### Przestrzeń poszukiwań

Przestrzenią poszukiwań jest zbiór wektorów o wartościach rzeczywistych. Wartości wektora reprezentują kąty w radianach między kolejnymi segmentami. W większości przypadków kąty pomiędzy segmentami były ograniczane do przedziału $[-\pi, \pi]$, w praktyce możliwe są jednak dowolne ich wartości. W przypadku pierwszego segmentu kąt jest mierzony do podłoża, dla kolejnych segmentów mierzone jest odchylenie od przedłużenia poprzedniego odcinka. Wszystkie kąty mierzone są przeciwnie do ruchu wskazówek zegara.

### Funkcja celu

Jako funkcję celu stosuję odległość euklidesową końca ostatniego segmentu od celu, podniesioną do kwadratu.

### Funkcje ograniczeń

Dla każdej przeszkody definiuję osobną funkcję ograniczeń, pozwalającą oszacować, w jakim stopniu ramię "nachodzi" na daną przeszkodę. Dla każdej kolizji obliczana jest wartość "kary", liczonej poprzez szacowanie pola mniejszej spośród figur, na które ramię dzieli daną przeszkodę. We wszystkich przypadkach oprócz 3. w obliczeniach rozważane są jedynie wierzchołki prostokąta i punkty przecięcia ramienia z bokami. Rozważane są następujące przypadki:
1. Ramię przecina dwa przeciwległe boki. Wówczas pole jest szacowane jako mniejsze z pól trapezów
1. Ramię przecina dwa sąsiednie boki. Pole jest szacowane jako pole powstałego trójkąta prostokątnego.
1. Ramię przecina dwa razy ten sam bok. Pole jest szacowane jako pole trójkąta o długości podstawy równej odległości między punktami przecięcia oraz wysokości równej maksymalnej odległości ramienia od rozważanego boku. Do długości podstawy dodawana jest niewielka wartość aby zapobiec zwróceniu zera w przypadku, gdy oba punkty przecięcia są równe.
1. Ramię kończy się wewnątrz prostokąta. Wówczas zwracane jest pole całego prostokąta

W przypadku gdy jeden osobnik przecina tę samą przeszkodę wiele razy, powyższe wartości są sumowane.

### Użyty algorytm

Do rozwiązania problemu stosuję algorytm IDEA w wersji dostosowanej do rozwiązywania problemów dynamicznych. W kolejnych krokach ewolucja jest rozpoczynana z populacją z poprzedniego kroku, w której część najsłabszych osobników zastąpiono losowymi imigrantami.

Algorytm opisany jest tutaj: https://www.researchgate.net/publication/225673364_Infeasibility_Driven_Evolutionary_Algorithm_for_Constrained_Optimization

Tutaj można przeczytać o jego stosowaniu do problemów dynamicznych: https://www.researchgate.net/publication/224472397_Performance_of_infeasibility_driven_evolutionary_algorithm_IDEA_on_constrained_dynamic_single_objective_optimization_problems

### Operatory ewolucyjne

Używane są operatory *simulated binary crossover* oraz *polynomial mutation*. W eksperymentach oprócz stałego prawdopodobieństwa mutacji testowałem stosowanie rożnego prawdopodobieństwa mutacji dla różnych segmentów, zgodnie z ideą, że zmiany bliżej podstawy mają większy wpływ na osobnika niż zmiany bliżej końca ramienia.

## Eksperymenty

### Implementacja

W każdym eksperymencie wczytywany jest zapisany wcześniej stan generatora liczb losowych, dzięki czemu eksperymenty powinny być powtarzalne. Ostatnia linijka w komórce zawierającej wywołanie algorytmu służy zapisaniu historii populacji oraz wartości funkcji celu w katalogu `/tmp`. Przy powtarzaniu prób można ją usunąć.

### Rozważane problemy i wyniki

1. Cel jest nieruchomy, występuje jedna przeszkoda poruszająca się w lewo. Algorytm zmuszony jest w pewnym momencie "przeskoczyć" na jej drugą stronę. Algorytm dobrze rozwiązuje ten problem pod warunkiem zastosowania losowych imigrantów.
2. Początek ramienia otoczony jest nieruchomymi przeszkodami okrążanymi przez cel. Przetestowałem zarówno stałe jak i zmienne prawdopodobieństwo mutacji dla kolejnych kątów, w obu przypadkach algorytm radził sobie dobrze.
3. Przeszkody i cel poruszają się ruchem jednostajnym w różnych kierunkach. Ramię musi przejść przez obecne w przeszkodach szczeliny. Ponownie testowałem zarówno stałe, jak i zmienne prawdopdobieństwo mutacji. W obu przypadkach algorytm zadział dobrze.
4. Ramię musi pokonać dwa rzędy przeszkód poruszające się w przeciwnych kierunkach, z wieloma dostępnymi szczelinami. Cel również porusza się ruchem jednostajnym. Ramię musi wielokrotnie zmieniać wykorzystywane szczeliny. Ponownie, problem został przez algorytm rozwiązany.
5. Trudniejsza wersja eksperymentu 3., algorytm musi pokonać dwie wąskie szczeliny przesuwające się w przeciwnych kierunkach. W tym eksperymencie właściwy dobór parametrów okazał się kluczowy. W szczególności, zastosowanie zmiennego prawdopodobieństwa mutacji znacznie podnosiło skuteczność algorytmu.
6. Tym razem algorytm musi pokonać jedną z dostępnych, długich i wąskich szczelin. Ponownie podejście korzystające ze zmiennego prawdopodobieństwa mutacji okazało się skuteczniejesze.
7. Algorytm ma do pokonania prosty labirynt. To zadanie wymagało dużej populacji oraz precyzyjnego dobrania parametrów algorytmu, udało się je jednak zrealizować. Problematyczne okazywały się połączenia między ścianami labiryntu, które tworzyły minima lokalne dla funkcji ograniczeń.
8. Ramię musi pokonać grupę losowo poruszających się przeszkód. Zadanie nie sprawiało problemów.
9. Ramię ma do pokonoania wiele przeszkód poruszających się w różnych kierunkach, cel znajduje się w dużej odległości. To zadanie jest istotnie trudniejsze od poprzednich, również ze względu na dużą liczbę segmentów (a więc duży wymiar przestrzeni poszukiwań). Tego zadania algorytmowi nie udało się rozwiązać.
10. Cel znajduje się w dużej odległości, tuż przed nim znajduje się poruszająca się ruchem jednostajnym przeszkoda ze szczeliną. Jest to znacznie uproszczona wersja poprzedniego problemu, tym razem algorytm znalazł rozwiązanie.
11. Cel przemieszcza się między trzema odzielonymi przeszkodami strefami. Ten problem wymaga niemal całkowitego rozprostownaia ramienia, co sprawiało istotną trudność. Niemniej jednak, przy zastosowaniu dużej liczby iteracji, udało się znaleźć rozwiązanie dość bliskie optymalnemu. Niewielka frakcja rozwiązań niepoprawnych umożliwiała sprawne roziązywanie problemu w kolejnych momentach.

## Wnioski

Algorytm radzi sobie dobrze z omijaniem pojedynczych przeszkód. Problemy może sprawiać natomiast ominięcie wielu przeszkód, w szczególności pokonywanie labiryntów wymaga dużych populacji i precyzyjnie dobranych parametrów. Występowały również problemy dla ramion złożonych z wielu segmentóœ w przypadku, gdy odległość od  celu nie zostawiała dużego marginesu błędu.

W większości problemów opłacalne było przechowywanie dużej frakcji rozwiązań niepoprawnych - w większości przypadków ominięcie przeszkód odbywało się poprzez "naprawienie" jednego z takich rozwiązań, podczas gdy rozwiązania poprawne pozostawały w minimach lokalnych.